<a href="https://colab.research.google.com/github/vizcayal/aha_moment/blob/main/grpo_training_qwen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Import Libraries

In [ ]:
import sys
import logging
modules = list(sys.modules.keys())
import torch
import re

#installing unsloth to fine-tune LLM models with Reinforcement Learning and Low-Rank Adapataion (LoRA)
!pip install unsloth
#installing vllm for fast inference, and reducing latency
!pip install vllm

In [ ]:
from unsloth import FastLanguageModel, PatchFastRL
from unsloth import is_bfloat16_supported             #check if bfloat16 is supported
from datasets import load_dataset, Dataset
from trl import GRPOConfig, GRPOTrainer
from vllm import SamplingParams

In [ ]:
PatchFastRL("GRPO", FastLanguageModel)                # apply patch for training LLMs with Group Relative Policy Optimization (GRPO) used in deepseek
max_seq_length = 512                                  # max sequencen length
lora_rank = 32                                        # dim for lora matrix

#load the Llama 3.1-8B model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
                                                    model_name = "meta-llama/meta-Llama-3.1-8B-Instruct",
                                                    max_seq_length = max_seq_length,
                                                    load_in_4bit = True,                                    # False for LoRA 16bit
                                                    fast_inference = True,                                  # Enable vLLM fast inference
                                                    max_lora_rank = lora_rank,
                                                    gpu_memory_utilization = 0.6,
                                                    )

model = FastLanguageModel.get_peft_model(                                                                   #Parameter efficient Fine-Tuning (peft)
                                        model,
                                        r = lora_rank,
                                        target_modules = [
                                                          "q_proj",
                                                          "k_proj",
                                                          "v_proj",
                                                          "o_proj",
                                                          "gate_proj",
                                                          "up_proj",
                                                          "down_proj",
                                                        ],
                                        lora_alpha = lora_rank,
                                        use_gradient_checkpointing = "unsloth",
                                        random_state = 3407,
                                        )

In [ ]:
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore
    return data # type: ignore

# Reward Functions


def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

# reward in case of digit
def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

# reward in case of following strictly the format
def strict_format_reward_func(completions, **kwargs) -> list[float]:
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

# reward in case of following strictly the format
def soft_format_reward_func(completions, **kwargs) -> list[float]:
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]


### Set up Parameters for GRPO training

In [ ]:
training_args = GRPOConfig(
                          use_vllm = True,
                          learning_rate = 5e-6,
                          adam_beta1 = 0.9,
                          adam_beta2 = 0.99,
                          weight_decay = 0.1,
                          warmup_ratio = 0.1,
                          lr_scheduler_type = 'cosine',
                          optim = 'paged_adamw_8bit',
                          logging_steps = 1,
                          bf16 = is_bfloat16_supported(),
                          fp16 = not is_bfloat16_supported(),
                          per_device_train_batch_size = 6,
                          gradient_accumulation_steps = 1,
                          num_generations = 6,
                          max_prompt_length = 256,
                          max_completion_length = 200,
                          max_steps = 250,
                          save_steps = 250,
                          max_grad_norm = 1,
                          report_to = 'none',
                          output_dir = 'outputs'
                          )
model.save_lora("model_grpo_lora")



```
# This is formatted as code
```

### Set up the GRPO Trainer

In [ ]:
dataset = get_gsm8k_question()
grpo_trainer = GRPOTrainer(
                          model = model,
                          processing_class = tokenizer,
                          reward_funcs =  [
                                            correctness_reward_f,
                                            int_reward_func,
                                            strict_format_reward_func,
                                            soft_format_reward_func,
                                            xmlcount_reward_func,
                                          ],
                          args = training_args,
                          train_dataset = dataset
                          )

grpo_trainer.train()

### Inference with no GRPO training

In [ ]:
text = tokenizer.apply_chat_template(
                                    [{'role': 'system', 'content': "calculate pi."}],
                                    tokenize = False,
                                    add_generation_prompt = True
                                    )
sampling_params = SamplingParams(
                                  temperature = 0.2,
                                  top_p = 0.95,
                                  max_tokes =1024
                                )

output = model.fast_generate(
                            [text],
                            sampling_params = sampling_params,
                            lora_request = None,
                            )[0].outputs[0].text

output

### Inference with grpo trained model

> Add blockquote



In [ ]:
reasoning_output = model.fast_generate(
                            [text],
                            sampling_params = sampling_params,
                            lora_request = model.load_lora('model_grpo_lora'),
                            )[0].outputs[0].text

reasoning_output